In [ ]:
"""
Fix amounts for older reservations when we didn't have invoice system
"""

In [1]:
import requests
import json
import os
from dictor import dictor
from threading import Thread
import queue


### step4
# Declare url and authentication with header secret FIXME protect this
###
headers = {'content-type': 'application/json', 'x-hasura-admin-secret': 'V3ntur4Pr0dH4sur4Tr4v3lC0re2o2o'}
url = 'https://core.venturatravel.org/v1/graphql'

def sendHasuraRequest(query, variables: dict={}):
    r = requests.post(url=url, json={'query': query, 'variables': variables}, headers=headers)
    return r.json()

In [2]:
def performRequest(query, ubjs, no_workers):
    class Worker(Thread):
        def __init__(self, request_queue):
            Thread.__init__(self)
            self.queue = request_queue
            self.results = []

        def run(self):
            while True:
                content = self.queue.get()
                if content == None:
                    break
                res = sendHasuraRequest(query, content)
                print("______________")
                print("Sending", content)
                print("Result", res)
                self.results.append(res)
                self.queue.task_done()

    # Create queue and add addresses
    q = queue.Queue()
    for ubj in ubjs:
        q.put(ubj)

    # Create workers and add tot the queue
    workers = []
    for _ in range(no_workers):
        worker = Worker(q)
        worker.start()
        workers.append(worker)
    # Workers keep working till they receive an empty string
    for _ in workers:
        q.put(None)
    # Join workers to wait till they finished
    for worker in workers:
        worker.join()

    # Combine results from all workers
    r = []
    for worker in workers:
        r.extend(worker.results)
    return r

In [13]:
queryReservations = """
query getReservations {
  reservation(where: {_and : [{ amount: { _is_null: true }}, { created_at: { _lte: "2010-12-31"}}]}, order_by: { created_at: asc}) {
    id
    amount
    amount_in_eur
    created_at
    main_departure {
      price
    }
  }
}
"""
res = sendHasuraRequest(queryReservations)
reservations = res['data']['reservation']
print(len(reservations))

178


In [12]:
toUpdate = []
for r in reservations:
    if r['main_departure'] == None: continue
    obj = {
        'id': r['id'],
        'amount': r['main_departure']['price']
    }
    toUpdate.append(obj)

In [10]:
len(toUpdate)

0

In [6]:
queryUpdateReservation = """
mutation updateReservationAmount($id: Int!, $amount: numeric) {
  update_reservation_by_pk(pk_columns: {id: $id}, _set: { amount: $amount }){
    id
  }
}
"""

In [7]:
results = performRequest(queryUpdateReservation, toUpdate, 6)
print("RESULTS")
print(results)

______________
Sending {'id': 107339, 'amount': 2140}
Result {'data': {'update_reservation_by_pk': {'id': 107339}}}
______________
Sending {'id': 107345, 'amount': 2240}
Result {'data': {'update_reservation_by_pk': {'id': 107345}}}
______________
Sending {'id': 107338, 'amount': 2790}
Result {'data': {'update_reservation_by_pk': {'id': 107338}}}
______________
Sending {'id': 107343, 'amount': 2490}
Result {'data': {'update_reservation_by_pk': {'id': 107343}}}
______________
Sending {'id': 107333, 'amount': 2490}
Result {'data': {'update_reservation_by_pk': {'id': 107333}}}
______________
Sending {'id': 107346, 'amount': 2650}
Result {'data': {'update_reservation_by_pk': {'id': 107346}}}
______________
Sending______________
Sending {'id': 107353, 'amount': 2340}
Result {'data': {'update_reservation_by_pk': {'id': 107353}}}
______________
Sending {'id': 107354, 'amount': 3090}______________
Sending {'id': 107352, 'amount': 2740}
Result {'data': {'update_reservation_by_pk': {'id': 107352}

______________
Sending {'id': 107455, 'amount': 2650}
Result {'data': {'update_reservation_by_pk': {'id': 107455}}}
______________
Sending {'id': 107458, 'amount': 2640}
Result {'data': {'update_reservation_by_pk': {'id': 107458}}}
______________
Sending {'id': 107461, 'amount': 2440}
Result {'data': {'update_reservation_by_pk': {'id': 107461}}}
______________
Sending {'id': 107472, 'amount': 2740}
Result {'data': {'update_reservation_by_pk': {'id': 107472}}}
______________
Sending {'id': 107467, 'amount': 2390}
Result {'data': {'update_reservation_by_pk': {'id': 107467}}}
______________
Sending {'id': 107474, 'amount': 2490}
Result {'data': {'update_reservation_by_pk': {'id': 107474}}}
______________
Sending {'id': 107476, 'amount': 2740}
Result {'data': {'update_reservation_by_pk': {'id': 107476}}}
______________
Sending {'id': 107477, 'amount': 2590}
Result {'data': {'update_reservation_by_pk': {'id': 107477}}}
______________
Sending {'id': 107478, 'amount': 2590}
Result {'data': {'

______________
Sending {'id': 107603, 'amount': 2680}
Result {'data': {'update_reservation_by_pk': {'id': 107603}}}
______________
Sending {'id': 107604, 'amount': 2790}
Result {'data': {'update_reservation_by_pk': {'id': 107604}}}
______________
Sending {'id': 107611, 'amount': 2750}
Result {'data': {'update_reservation_by_pk': {'id': 107611}}}
______________
Sending {'id': 107612, 'amount': 2640}
Result {'data': {'update_reservation_by_pk': {'id': 107612}}}
______________
Sending {'id': 107616, 'amount': 2790}
Result {'data': {'update_reservation_by_pk': {'id': 107616}}}
______________
Sending {'id': 107606, 'amount': 2240}
Result {'data': {'update_reservation_by_pk': {'id': 107606}}}
______________
Sending {'id': 107618, 'amount': 2590}
Result {'data': {'update_reservation_by_pk': {'id': 107618}}}
______________
Sending {'id': 107598, 'amount': 2680}
Result {'data': {'update_reservation_by_pk': {'id': 107598}}}______________
Sending {'id': 107605, 'amount': 2490}
Result 
{'data': {'

______________
Sending {'id': 107731, 'amount': 2990}
Result {'data': {'update_reservation_by_pk': {'id': 107731}}}
______________
Sending {'id': 107730, 'amount': 2590}
Result {'data': {'update_reservation_by_pk': {'id': 107730}}}
______________
Sending {'id': 107737, 'amount': 2640}
Result {'data': {'update_reservation_by_pk': {'id': 107737}}}
______________
Sending {'id': 107738, 'amount': 2990}
Result {'data': {'update_reservation_by_pk': {'id': 107738}}}
______________
Sending {'id': 107735, 'amount': 2750}
Result {'data': {'update_reservation_by_pk': {'id': 107735}}}
______________
Sending {'id': 107739, 'amount': 2750}
Result {'data': {'update_reservation_by_pk': {'id': 107739}}}
______________
Sending {'id': 107742, 'amount': 2690}
Result {'data': {'update_reservation_by_pk': {'id': 107742}}}
______________
Sending {'id': 107736, 'amount': 2690}
Result {'data': {'update_reservation_by_pk': {'id': 107736}}}
____________________________
Sending {'id': 107743, 'amount': 2590}
Resu

______________
Sending {'id': 107870, 'amount': 2640}
______________
Sending Result {'data': {'update_reservation_by_pk': {'id': 107870}}}
{'id': 107888, 'amount': 1740}
Result {'data': {'update_reservation_by_pk': {'id': 107888}}}
______________
Sending {'id': 107883, 'amount': 2440}
Result {'data': {'update_reservation_by_pk': {'id': 107883}}}
____________________________
Sending {'id': 107880, 'amount': 2340}
Result 
{'data': {'update_reservation_by_pk': {'id': 107880}}}
______________
Sending Sending {'id': 107877, 'amount': 2650}
Result{'id': 107875, 'amount': 2440}
Result {'data': {'update_reservation_by_pk': {'id': 107875}}}
 {'data': {'update_reservation_by_pk': {'id': 107877}}}
______________
Sending {'id': 107894, 'amount': 2390}
______________
Sending {'id': 107887, 'amount': 2440}
Result {'data': {'update_reservation_by_pk': {'id': 107887}}}
______________
Sending {'id': 107892, 'amount': 2690}
Result {'data': {'update_reservation_by_pk': {'id': 107894}}}
______________
Sen

...

______________
Sending {'id': 118352, 'amount': 1990}
Result {'data': {'update_reservation_by_pk': {'id': 118352}}}
______________
Sending {'id': 118355, 'amount': 2399}
Result {'data': {'update_reservation_by_pk': {'id': 118355}}}
______________
Sending {'id': 118358, 'amount': 4099}
Result {'data': {'update_reservation_by_pk': {'id': 118358}}}
______________
Sending {'id': 118360, 'amount': 3799}
Result {'data': {'update_reservation_by_pk': {'id': 118360}}}
______________
Sending {'id': 118359, 'amount': 3599}
Result {'data': {'update_reservation_by_pk': {'id': 118359}}}
______________
Sending {'id': 118363, 'amount': 3390}
Result {'data': {'update_reservation_by_pk': {'id': 118363}}}
______________
Sending {'id': 118362, 'amount': 3599}
Result {'data': {'update_reservation_by_pk': {'id': 118362}}}
______________
Sending {'id': 118372, 'amount': 2399}
Result {'data': {'update_reservation_by_pk': {'id': 118372}}}
______________
Sending {'id': 118361, 'amount': 2899}
Result {'data': {'

______________
Sending {'id': 118477, 'amount': 3799}
Result {'data': {'update_reservation_by_pk': {'id': 118477}}}
______________
Sending {'id': 118478, 'amount': 3399}
Result {'data': {'update_reservation_by_pk': {'id': 118478}}}
______________
Sending {'id': 118484, 'amount': 2799}
Result {'data': {'update_reservation_by_pk': {'id': 118484}}}
______________
Sending {'id': 118481, 'amount': 2899}
Result {'data': {'update_reservation_by_pk': {'id': 118481}}}
______________
Sending {'id': 118483, 'amount': 2799}
Result {'data': {'update_reservation_by_pk': {'id': 118483}}}
______________
Sending {'id': 118482, 'amount': 2599}
Result {'data': {'update_reservation_by_pk': {'id': 118482}}}
______________
______________
Sending {'id': 118485, 'amount': 3599}
Result {'data': {'update_reservation_by_pk': {'id': 118485}}}Sending
 {'id': 118487, 'amount': 3399}
Result {'data': {'update_reservation_by_pk': {'id': 118487}}}
______________
Sending {'id': 118488, 'amount': 3899}
Result {'data': {'

______________
Sending {'id': 118591, 'amount': 4799}
Result {'data': {'update_reservation_by_pk': {'id': 118591}}}
______________
Sending {'id': 118593, 'amount': 4799}
Result {'data': {'update_reservation_by_pk': {'id': 118593}}}
______________
Sending {'id': 118596, 'amount': 3899}
Result {'data': {'update_reservation_by_pk': {'id': 118596}}}
______________
Sending {'id': 118594, 'amount': 3299}
Result {'data': {'update_reservation_by_pk': {'id': 118594}}}
______________
Sending {'id': 118610, 'amount': 3299}
Result {'data': {'update_reservation_by_pk': {'id': 118610}}}
______________
Sending {'id': 118598, 'amount': 3299}
Result {'data': {'update_reservation_by_pk': {'id': 118598}}}
______________
Sending {'id': 118601, 'amount': 3299}
______________
Sending {'id': 118602, 'amount': 3299}
Result______________
Sending {'id': 118604, 'amount': 4599}
Result {'data': {'update_reservation_by_pk': {'id': 118604}}}
______________
Sending  {'data': {'update_reservation_by_pk': {'id': 11860

Result {'data': {'update_reservation_by_pk': {'id': 118683}}}
______________
Sending {'id': 118689, 'amount': 3699}
Result {'data': {'update_reservation_by_pk': {'id': 118689}}}
______________
Sending {'id': 118687, 'amount': 3390}
Result {'data': {'update_reservation_by_pk': {'id': 118687}}}
______________
Sending {'id': 119043, 'amount': 3290}
Result {'data': {'update_reservation_by_pk': {'id': 119043}}}
______________
Sending {'id': 118698, 'amount': 3690}
Result {'data': {'update_reservation_by_pk': {'id': 118698}}}
______________
Sending {'id': 118695, 'amount': 3999}
Result {'data': {'update_reservation_by_pk': {'id': 118695}}}
______________
Sending {'id': 118688, 'amount': 3699}
Result {'data': {'update_reservation_by_pk': {'id': 118688}}}
______________
Sending {'id': 118693, 'amount': 3290}
Result {'data': {'update_reservation_by_pk': {'id': 118693}}}
______________
Sending {'id': 118697, 'amount': 4199}
Result ______________{'data': {'update_reservation_by_pk': {'id': 118697

____________________________
Sending {'id': 118810, 'amount': 3999}
Result {'data': {'update_reservation_by_pk': {'id': 118810}}}

Sending {'id': 118812, 'amount': 3999}
Result {'data': {'update_reservation_by_pk': {'id': 118812}}}
______________
Sending {'id': 118814, 'amount': 2999}
Result {'data': {'update_reservation_by_pk': {'id': 118814}}}
______________
Sending {'id': 118809, 'amount': 3299}
Result {'data': {'update_reservation_by_pk': {'id': 118809}}}
______________
Sending {'id': 118815, 'amount': 3699}
Result {'data': {'update_reservation_by_pk': {'id': 118815}}}
______________
Sending {'id': 118822, 'amount': 2999}
Result {'data': {'update_reservation_by_pk': {'id': 118822}}}
______________
Sending {'id': 118831, 'amount': 3999}
Result {'data': {'update_reservation_by_pk': {'id': 118831}}}
______________
Sending {'id': 118824, 'amount': 3799}
Result {'data': {'update_reservation_by_pk': {'id': 118824}}}
______________
Sending {'id': 118829, 'amount': 4499}
Result {'data': {'